# **Section 1: Setup and Imports** <a id="1"></a>

## **Importing necessary libraries**

In [1]:
import os
import sys
import uuid

## **Import Shared Functions From Project Folder** <a id="1.1"></a>
We will import the shared functions from the project folder that we built for IT Consultant Report Reviewer with ReAct. This is done to keep the notebook clean and organized. The functions are used for data loading, preprocessing, and evaluation.

In [2]:
# ✅ Adjust the path if your notebook is in /notebooks and src/ is a sibling
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_root))

# Core OpenAI API
from src.models.openai_interface import call_openai_with_tracking

# ReAct reasoning agent and tools
from src.server.react_agent import (
    ReActConsultantAgent,
    run_react_loop_check_withTool,
    dispatch_tool_action, 
    select_best_tool_with_llm,
    run_react_loop_for_rfp_eval
)

# Proposal orchestration (ToT + ReAct hybrid)
#from src.server.report_review_runner import (
#    summarize_and_score_section
#)

# Prompt builders and tool descriptions
from src.server.prompt_builders import (
    build_tool_hints,
    format_tool_catalog_for_prompt
)

# Proposal scoring and analysis
from src.models.scoring import summarize_and_score_section

# LLM-based section tools (for evaluation scoring)
from src.models.section_tools_llm import (
    should_cite,
    auto_fill_gaps_with_research,
    upgrade_section_with_research,
    make_text_coherent,
    generate_final_summary,
    format_upgraded_sections
)

# File loading utility (later use for multi-proposal)
from src.utils.file_loader import load_report_text_from_file

# Text processing (optional for parsing requirements/proposals)
#from src.utils.text_processing import (
#    split_report_into_sections,
#    map_section_to_canonical,
#    guess_canonical_section_with_llm
#)

# Visualization (optional for tool analysis)
from src.utils.visualization import (
    print_tool_usage,
    plot_tool_usage
)

# Exporting to markdown + PDF
#from src.utils.export_utils import (
#    export_report_to_markdown,
#    export_report_to_markdown_and_pdf,
#    show_agent_memory
#)

# Basic tool logic
from src.utils.tools.tools_basic import (
    check_guideline_dynamic,
    keyword_match_in_section,
    check_timeline_feasibility,
    search_report,
    highlight_missing_sections,
    check_alignment_with_goals,
    compare_with_other_section,
    generate_client_questions
)

# Web tools
from src.utils.tools.tools_web import (
    search_web,
    search_serpapi,
    search_wikipedia,
    search_arxiv,
    should_search_arxiv
)

# NLP tools
from src.utils.tools.tools_nlp import (
    check_for_jargon,
    check_readability,
    analyze_tone_textblob,
    extract_named_entities
)

# Reasoning tools
from src.utils.tools.tools_reasoning import (
    pick_tool_by_intent,
    pick_tool_by_intent_fuzzy,
    categorize_tools_by_priority,
    analyze_math_question,
)

# LLM-based tools (LLM - thought generator)
from src.models.tot_agent import generate_thoughts_openai

from src.utils.tools.tool_embeddings import (
    build_tool_embeddings, 
    suggest_tools_by_embedding
)

from src.models.openai_embeddings import get_openai_embedding

from src.utils.tools.tool_catalog import tool_catalog


## **Text Embeddings of Tool Catalog** <a id="1.1"></a>
First, we need to load the tool catalog and create text embeddings for each tool. This will allow us to compare the user's query with the tools available in the catalog.

In [3]:
# One-time setup
tool_embeddings = build_tool_embeddings(tool_catalog)


✅ Loaded cached tool embeddings.


# **Section 2: Functions** <a id="1"></a>

Only new ones built for RFP Evaluation (ToT+ReACT Hybrid)

In [4]:
# --- Tree Node Class ---


class TreeNode:
    """
    Represents a node in the Tree of Thought (ToT) reasoning process.

    Purpose:
    This class models a single node in the reasoning tree, which includes a thought, its score, a reference to its parent node, and its children nodes. It provides methods to traverse the tree and retrieve the reasoning path.

    Parameters:
    - thought (str): The thought or reasoning step represented by this node.
    - score (float): The score or evaluation of this thought. Default is 0.
    - parent (TreeNode): The parent node of this node. Default is None.

    Functions:
    - __init__: Initializes the node with a thought, score, parent, and generates a unique ID.
    - path: Retrieves the reasoning path from the root to this node.

    Workflow:
    1. Initializes a unique ID for the node using UUID.
    2. Stores the thought, score, and parent node.
    3. Maintains a list of child nodes.
    4. Provides a `path` method to retrieve the reasoning path from the root to this node.

    Returns:
    - TreeNode: An instance of the TreeNode class representing a reasoning step in the ToT process.
    """

    def __init__(self, thought, score=0, parent=None):
        self.id = str(uuid.uuid4())
        self.thought = thought
        self.score = score
        self.parent = parent
        self.children = []

    def path(self):
        node, result = self, []
        while node:
            result.append(node.thought)
            node = node.parent
        return list(reversed(result))


# --- Basic ToT Agent ---


class SimpleToTAgent:
    """
    Represents a simple Tree of Thought (ToT) reasoning agent.

    Purpose:
    This class models a basic agent that uses a Tree of Thought approach to evaluate a given section of a proposal against a specific criterion. It generates thoughts, evaluates them, and selects the best reasoning path.

    Parameters:
    - llm (callable): A language model function that generates thoughts based on a prompt.
    - scorer (callable): A scoring function that evaluates the quality of a thought.
    - beam_width (int): The number of top thoughts to keep at each depth. Default is 2.
    - max_depth (int): The maximum depth of the reasoning tree. Default is 2.

    Functions:
    - __init__: Initializes the agent with the provided LLM, scorer, beam width, and maximum depth.
    - generate_thoughts: Generates thoughts for a given section and criterion using the LLM.
    - evaluate_and_select: Evaluates and selects the top thoughts based on their scores.
    - run: Constructs a reasoning tree by iteratively expanding thoughts up to the maximum depth and returns the best reasoning path and its score.

    Workflow:
    1. Initializes the agent with the provided LLM, scorer, beam width, and maximum depth.
    2. Generates thoughts for a given section and criterion using the LLM.
    3. Evaluates and selects the top thoughts based on their scores.
    4. Constructs a reasoning tree by iteratively expanding thoughts up to the maximum depth.
    5. Returns the best reasoning path and its score.

    Returns:
    - SimpleToTAgent: An instance of the SimpleToTAgent class capable of performing Tree of Thought reasoning.
    """

    def __init__(self, llm, scorer, beam_width=2, max_depth=2):
        self.llm = llm
        self.scorer = scorer
        self.beam_width = beam_width
        self.max_depth = max_depth

    def generate_thoughts(self, section, criterion, parent_node):
        base_prompt = f"""
    You are evaluating a technology vendor proposal for the criterion: **{criterion}**.

    Proposal excerpt:
    \"\"\"
    {section}
    \"\"\"

    Current reasoning path: {" -> ".join(parent_node.path()[1:]) if parent_node.thought != "ROOT" else "(start)"}

    What are the next 3 useful thoughts or questions that could help you assess the proposal based on this criterion?
    Respond with a numbered list.
    """
        response = self.llm(base_prompt)

        # Parse the LLM output into individual thoughts
        thoughts = [
            t.strip("123. ").strip("-• ") for t in response.split("\n") if t.strip()
        ]  # strips #s, -, and parses thoughs by line break
        return thoughts[:3]

    def evaluate_and_select(self, nodes):
        for node in nodes:
            node.score = self.scorer(node.thought)
        return sorted(nodes, key=lambda n: n.score, reverse=True)[: self.beam_width]

    def run(self, section, criterion):
        root = TreeNode("ROOT")
        frontier = [root]

        for depth in range(self.max_depth):
            print(
                f"\n🔁 Expanding depth {depth + 1}/{self.max_depth} — Frontier size: {len(frontier)}"
            )
            next_frontier = []

            for node in frontier:
                thoughts = self.generate_thoughts(section, criterion, node)
                print(f"💡 Thoughts generated from: '{node.thought}'")
                print("\n".join(f"  → {t}" for t in thoughts))

                if not thoughts:
                    print("⚠️ No thoughts returned. Skipping.")
                    continue

                child_nodes = [TreeNode(thought=t, parent=node) for t in thoughts]
                top_children = self.evaluate_and_select(child_nodes)

                for child in top_children:
                    print(f"✅ Selected: {child.thought} (score: {child.score})")

                node.children.extend(top_children)
                next_frontier.extend(top_children)

            frontier = next_frontier

            if not frontier:
                print("⚠️ No more frontier nodes. Stopping early.")
                break

        if frontier:
            best_leaf = max(
                frontier, key=lambda n: n.score
            )  # Get the best leaf node based on score
            return {
                "criterion": criterion,
                "score": best_leaf.score,
                "reasoning_path": best_leaf.path()[1:],  # skip 'ROOT'
            }
        else:
            return {
                "criterion": criterion,
                "score": 0,
                "reasoning_path": ["No valid thoughts generated."],
            }

In [5]:
def score_thought_with_openai(thought, criterion, section, model="gpt-3.5-turbo"):
    """
    Purpose:
    Scores a thought generated by an AI agent for assessing a technology proposal based on its relevance, clarity, and usefulness.

    Parameters:
    - thought (str): The thought or reasoning step to be evaluated.
    - criterion (str): The evaluation criterion against which the thought is being assessed.
    - section (str): The proposal section being evaluated.
    - model (str): The name of the OpenAI model to use for scoring. Default is "gpt-3.5-turbo".

    Workflow:
    1. Constructs a prompt that includes the proposal section, evaluation criterion, and the thought to be scored.
    2. Sends the prompt to the OpenAI API using the `call_openai_with_tracking` function.
    3. Prints the thought being scored and the response from the LLM.
    4. Attempts to parse the response as an integer score between 1 and 10.
    5. If parsing fails, defaults to a fallback score of 5.

    Returns:
    - int: The score assigned to the thought, ranging from 1 to 10.
    """
    prompt = f"""
    You are evaluating a thought generated by an AI agent for assessing a technology proposal.

    Use this rubric to assign a score from 1 to 10:

    - 9–10: Insightful, highly relevant, and clearly advances evaluation.
    - 7–8: Solid, useful thought, but not exceptionally insightful.
    - 4–6: Somewhat helpful, but vague or redundant.
    - 1–3: Not useful, unclear, or irrelevant.

    Proposal:
    \"\"\"
    {section}
    \"\"\"

    Criterion: {criterion}
    Thought: "{thought}"

    Respond with a single number only, from 1 to 10.
    """
    messages = [{"role": "user", "content": prompt}]
    response = call_openai_with_tracking(messages, model=model, temperature=0)

    print("\n🧠 Scoring Thought:")
    print(f"→ {thought}")
    print(f"📩 LLM Response: {response}")

    try:
        score = int(response.strip())
        print(f"✅ Parsed Score: {score}/10")
        return score
    except ValueError:
        print("⚠️ Failed to parse score, using fallback score = 5")
        return 5

In [6]:
def format_evaluation_report(results):
    """
    Purpose:
    Formats a full evaluation report from a list of Tree of Thought (ToT) results into a markdown string.

    Parameters:
    - results (list of dict): A list of evaluation results, where each result is a dictionary containing:
        - criterion (str): The evaluation criterion.
        - score (int): The score assigned to the criterion (1–10).
        - reasoning_path (list of str): The reasoning path (thoughts) generated during the evaluation.

    Workflow:
    1. Initializes a markdown report string with a title.
    2. Iterates through the `results` list:
        - Extracts the criterion, score, and reasoning path for each result.
        - Appends the criterion, score, and reasoning path to the report in markdown format.
        - Accumulates the total score for calculating the average.
    3. Computes the average score across all criteria.
    4. Appends the overall average score to the report.
    5. Returns the formatted markdown report string.

    Returns:
    - str: A markdown-formatted string representing the evaluation report.
    """
    report = "# 📄 Proposal Evaluation Report\n\n"
    total_score = 0

    for result in results:
        criterion = result["criterion"]
        score = result["score"]
        thoughts = result["reasoning_path"]
        total_score += score

        report += f"## {criterion}\n"
        report += f"**Score**: {score}/10\n\n"
        report += "**Reasoning Path:**\n\n"
        for i, t in enumerate(thoughts, 1):
            report += f"{i}. {t}\n\n"
        report += "\n"

    avg_score = round(total_score / len(results), 2)
    report += f"---\n\n**🧾 Overall Average Score:** {avg_score}/10\n"

    return report


In [7]:
def export_report_to_markdown_tot(report_text, filename="proposal_evaluation_report.md"):
    """
    Purpose:
    Exports a Tree of Thought (ToT) evaluation report to a markdown (.md) file.

    Parameters:
    - report_text (str): The content of the evaluation report in markdown format.
    - filename (str): The name of the markdown file to save the report to. Default is "proposal_evaluation_report.md".

    Workflow:
    1. Opens the specified file in write mode with UTF-8 encoding.
    2. Writes the provided `report_text` to the file.
    3. Prints a confirmation message with the file path.

    Returns:
    - None: This function does not return any value. It saves the report to a file.
    """
    with open(filename, "w", encoding="utf-8") as f:
        f.write(report_text)
    print(f"✅ Markdown report saved to: {filename}")


In [8]:
async def export_tot_report_to_markdown_and_pdf(
    report_md_text,
    markdown_file="proposal_evaluation_report.md",
    pdf_file="proposal_evaluation_report.pdf"
):
    """
    Purpose:
    Exports a Tree of Thought (ToT) evaluation report (provided as a markdown string) to both `.md` and `.pdf` formats.

    Parameters:
    - report_md_text (str): The content of the evaluation report in markdown format.
    - markdown_file (str): The name of the markdown file to save the report to. Default is "proposal_evaluation_report.md".
    - pdf_file (str): The name of the PDF file to save the report to. Default is "proposal_evaluation_report.pdf".

    Workflow:
    1. Ensures the output directory exists (`../outputs/` relative to the current working directory).
    2. Saves the markdown content to a `.md` file in the output directory.
    3. Converts the markdown content to HTML using the `markdown` library.
    4. Wraps the HTML content in a basic HTML template with styling.
    5. Saves the HTML content to a temporary `.html` file in the output directory.
    6. Uses Playwright to render the HTML file and export it as a PDF.
    7. Handles any exceptions during the PDF export process and logs an error message if it fails.

    Returns:
    - None: This function does not return any value. It saves the report to `.md` and `.pdf` files in the output directory.
    """


In [9]:
def score_proposal_content_with_llm(proposal, criterion, top_thoughts=None, model="gpt-3.5-turbo"):
    """
    Purpose:
    Scores how well the proposal meets a specific RFP criterion, optionally guided by Tree of Thought (ToT) thoughts.

    Parameters:
    - proposal (str): The text of the vendor proposal being evaluated.
    - criterion (str): The evaluation criterion against which the proposal is being assessed.
    - top_thoughts (list of str, optional): A list of key thoughts or considerations generated during the evaluation process. Default is None.
    - model (str): The name of the OpenAI model to use for scoring. Default is "gpt-3.5-turbo".

    Workflow:
    1. Constructs a prompt that includes the proposal text, evaluation criterion, and optionally, key thoughts.
    2. Sends the prompt to the OpenAI API using the `call_openai_with_tracking` function.
    3. Parses the response to extract the score and explanation.
    4. If parsing fails, defaults to a fallback score of 5 and a generic explanation.

    Returns:
    - tuple: A tuple containing:
        - score (int): The score assigned to the proposal, ranging from 1 to 10.
        - explanation (str): The reasoning behind the assigned score.
    """
    thoughts_text = ""
    if top_thoughts:
        thoughts_text = "\nHere are some important considerations:\n" + "\n".join(f"- {t}" for t in top_thoughts)

    prompt = f"""
You are evaluating a vendor proposal on the criterion: **{criterion}**.

Proposal:
\"\"\"
{proposal}
\"\"\"
{thoughts_text}

Based on the proposal and the evaluation criteria above, assign a score from 1 to 10.

Respond in this format:
Score: X
Explanation: (why this score)
"""

    messages = [{"role": "user", "content": prompt}]
    response = call_openai_with_tracking(messages, model=model, temperature=0)

    try:
        lines = response.strip().split("\n")
        score_line = next((l for l in lines if "Score:" in l), "Score: 5")
        explanation = next((l for l in lines if "Explanation:" in l), "Explanation: No explanation found.")
        score = int(score_line.split(":")[1].strip())
        explanation = explanation.split(":", 1)[1].strip()
        return score, explanation
    except Exception as e:
        print(f"⚠️ Failed to parse score: {str(e)}")
        return 5, "Failed to parse explanation"


In [10]:
def evaluate_proposal(proposal_text, rfp_criteria, model="gpt-3.5-turbo"):
    """
    Purpose:
    Performs a full hybrid evaluation for a single proposal across all criteria. 
    Combines Tree of Thought (ToT) reasoning, ReAct tool-based analysis, proposal scoring, 
    overall score computation, and SWOT assessment generation.

    Parameters:
    - proposal_text (str): The text of the vendor proposal being evaluated.
    - rfp_criteria (list of str): A list of evaluation criteria to assess the proposal against.
    - model (str): The name of the OpenAI model to use for LLM-based reasoning and scoring. Default is "gpt-3.5-turbo".

    Workflow:
    1. Initializes an empty list to store evaluation results.
    2. Iterates through each criterion in `rfp_criteria`:
        - Runs a Tree of Thought (ToT) reasoning process to generate thoughts and reasoning paths.
        - Executes a ReAct reasoning loop to select and apply tools for analysis.
        - Scores the proposal based on the generated thoughts and tool results using an LLM.
        - Collects the criterion-level score, explanation, and tool usage into the results.
    3. Computes the overall score as a simple average of all criterion-level scores.
    4. Generates an evaluation summary string based on the results.
    5. Creates a SWOT assessment (Strengths, Weaknesses, Opportunities, Threats) using an LLM.
    6. Returns the evaluation results, overall score, and SWOT summary.

    Returns:
    - tuple: A tuple containing:
        - results (list of dict): Criterion-level evaluation results, including scores, reasoning paths, and tool usage.
        - overall_score (float): The overall average score across all criteria.
        - swot_summary (str): A SWOT assessment of the proposal.
    """
    results = []

    for criterion in rfp_criteria:
        print(f"\n=== Evaluating: {criterion} ===")

        # Step 1: Run ToT for reasoning path to generate thoughts (questions) by criterion
        tot_agent = SimpleToTAgent(
            llm=generate_thoughts_openai,
            scorer=lambda t: score_thought_with_openai(t, criterion, proposal_text),
            beam_width=2,
            max_depth=2
        )
        result = tot_agent.run(section=proposal_text, criterion=criterion)
        top_thoughts = result["reasoning_path"]

        # Step 2–4: Run ReAct loop using ToT thoughts and embedding-aware tool selection
        react_agent = ReActConsultantAgent(section_name=criterion, section_text=proposal_text)
        report_sections = {"Proposal": proposal_text}
        #react_agent.report_section = proposal_text
        tool_history = run_react_loop_for_rfp_eval(
            agent=react_agent,
            criterion=criterion,
            proposal_text=proposal_text,
            thoughts=top_thoughts,
            tool_embeddings=tool_embeddings,
            report_sections=report_sections,
            max_steps=4
        )

        # Step 5: Extract ReAct Thought->Act->Observe history for context in proposal evaluation
        triggered_tools = [
            {
                "tool": step["action"],
                "result": step["observation"],
                "thought": step["thought"]
            }
            for step in tool_history
            if step.get("action") not in ["summarize", "ask_question"]
        ]
        result["react_thoughts"] = [step["thought"] for step in tool_history]
        
        # Combine ToT and ReAct thoughts
        all_thoughts = top_thoughts + result["react_thoughts"]

        # Step 6: Score proposal using LLM with ToT thoughts and tool results
        proposal_score, explanation = score_proposal_content_with_llm_and_tools(
            proposal=proposal_text,
            criterion=criterion,
            top_thoughts=all_thoughts,
            triggered_tools=triggered_tools,
            model=model
        )

        # Step 5: Add everything to result object
        result["proposal_score"] = proposal_score
        result["proposal_explanation"] = explanation
        result["triggered_tools"] = triggered_tools
        results.append(result)

    # Step 6: Compute overall weighted average score (simple average for now)
    total = sum(r["proposal_score"] for r in results)
    overall_score = round(total / len(results), 2)

    # Generate the evaluation summary 
    eval_summary = ''.join(
        f"- {r['criterion']}: Score {r['proposal_score']}/10 – {r['proposal_explanation']}\n"
        for r in results
    )

    # Step 7: Generate SWOT summary using LLM
    swot_prompt = f"""
You are summarizing a vendor proposal based on the following criterion-level evaluations:

{eval_summary}

Generate a SWOT assessment (Strengths, Weaknesses, Opportunities, Threats) for this proposal.
"""
    messages = [{"role": "user", "content": swot_prompt}]
    swot_summary = call_openai_with_tracking(messages, model=model)

    return results, overall_score, swot_summary


In [11]:
def print_proposal_evaluation(results, overall_score, swot_summary):
    """
    Nicely prints the full evaluation for a proposal, including:
    - Criterion-level scores and reasoning
    - ToT thoughts
    - Tools used and results
    - Final overall score
    - SWOT summary
    """
    print("\n====================")
    print("📊 PROPOSAL EVALUATION")
    print("====================\n")

    for result in results:
        print(f"🔹 Criterion: {result['criterion']}")
        print(f"📈 Score: {result['proposal_score']}/10")
        print(f"🧠 Thoughts:")
        for t in result["reasoning_path"]:
            print(f"   • {t}")
        print("🛠️ Tools Used:")
        for tool in result["triggered_tools"]:
            print(f"   • {tool['tool']}: {tool['result']}")
        print(f"🗣️ Explanation: {result['proposal_explanation']}")
        print("\n--------------------\n")

    print(f"✅ OVERALL SCORE: {overall_score}/10\n")
    print("📋 SWOT ASSESSMENT:\n")
    print(swot_summary)
    print("\n====================\n")


In [12]:
def score_proposal_content_with_llm_and_tools(proposal, criterion, top_thoughts=None, triggered_tools=None, model="gpt-3.5-turbo"):
    """
    Purpose:
    Uses an LLM to score a vendor proposal based on Tree of Thought (ToT) evaluation thoughts and insights from triggered tools.

    Parameters:
    - proposal (str): The text of the vendor proposal being evaluated.
    - criterion (str): The evaluation criterion against which the proposal is being assessed.
    - top_thoughts (list of str, optional): A list of key thoughts or considerations generated during the evaluation process. Default is None.
    - tool_observations (str, optional): Observations or insights from tools that may influence the scoring. Default is None.
    - triggered_tools (list of dict, optional): A list of tool results, where each dict contains:
        - 'tool' (str): The name of the tool.
        - 'result' (str): The output or insight generated by the tool. Default is None.
    - model (str): The name of the OpenAI model to use for scoring. Default is "gpt-3.5-turbo".

    Workflow:
    1. Initializes `triggered_tools` as an empty list if not provided.
    2. Formats the tool results into a string for inclusion in the LLM prompt.
    3. Formats the top thoughts into a string for inclusion in the LLM prompt.
    4. Constructs a prompt that includes the proposal text, evaluation criterion, top thoughts, and tool insights.
    5. Sends the prompt to the OpenAI API using the `call_openai_with_tracking` function.
    6. Parses the response to extract the score and explanation.
    7. If parsing fails, defaults to a fallback score of 5 and a generic explanation.

    Returns:
    - tuple: A tuple containing:
        - score (int): The score assigned to the proposal, ranging from 1 to 10.
        - explanation (str): The reasoning behind the assigned score.
    """
    if triggered_tools is None:
        triggered_tools = []

    # Format tool output
    tool_insights = "\n".join([
        f"{tool['tool']}: {tool['result']}" for tool in triggered_tools
    ]) if triggered_tools else ""

    # Format top thoughts
    thoughts_text = ''.join(f"- {t}\n" for t in top_thoughts or [])

    prompt = f"""
You are evaluating a vendor proposal on the criterion: **{criterion}**.

Proposal:
\"\"\"
{proposal}
\"\"\"

Top Evaluation Thoughts:
{thoughts_text}

Tool Analysis:
{tool_insights}

Now, assign a score from 1 to 10 for how well the proposal addresses this criterion.
Respond in this format:
Score: X
Explanation: (your reasoning)
"""
    messages = [{"role": "user", "content": prompt}]
    response = call_openai_with_tracking(messages, model=model, temperature=0)

    try:
        lines = response.strip().split("\n")
        score_line = next((l for l in lines if "Score:" in l), "Score: 5")
        explanation = next((l for l in lines if "Explanation:" in l), "Explanation: No explanation found.")
        score = int(score_line.split(":")[1].strip())
        explanation = explanation.split(":", 1)[1].strip()
        return score, explanation
    except Exception as e:
        print(f"⚠️ Failed to parse score: {str(e)}")
        return 5, "Failed to parse explanation"


# **Section 3: Load RFP Data** <a id="3"></a>

In [13]:
# --- Use Case ---
use_case = """
A public sector organization is seeking a new cloud-based Electronic Health Record (EHR) system. 
They've issued an RFP to multiple technology vendors. Each proposal must address:
- Functional fit to healthcare workflows
- Technical architecture
- Cost structure
- Implementation timeline
- Vendor experience
- Risk management

As a first step, we want our AI agent to evaluate a single proposal using Tree of Thought reasoning.
"""

# --- Evaluation Criteria ---
rfp_criteria = [
    "Solution Fit",
    "Technical Architecture",
    "Cost",
    "Implementation Timeline",
    "Vendor Experience",
    "Risk Management",
]

# --- Sample Proposal (Vendor A) ---
proposal_a = """
Vendor A proposes a modular, cloud-native EHR platform with configurable workflows for clinics and hospitals. 
The platform supports FHIR interoperability, role-based access, and integration with provincial health registries.
The implementation will follow a phased approach over 18 months. Initial go-live includes primary care, with 
specialty modules added later.

The vendor has delivered similar solutions in two provinces and offers a dedicated implementation team. 
They note some assumptions: client to provide system integration support, legacy data migration, and end-user training. 
Pricing is tiered by user volume and includes hosting, support, and upgrades.
"""

print("✅ Use Case and Sample Proposal loaded.")

✅ Use Case and Sample Proposal loaded.


# **Section 4: Evaluate Proposals** <a id="4"></a>
**Run Hybrid ToT + ReAct Model**

In [14]:
# Step 1: Run evaluation
results_a, overall_score_a, swot_summary_a = evaluate_proposal(proposal_a, rfp_criteria)


=== Evaluating: Solution Fit ===

🔁 Expanding depth 1/2 — Frontier size: 1
🔢 Prompt: 210 tokens | Completion: 80 tokens | Cost: $0.0004 USD
💡 Thoughts generated from: 'ROOT'
  → How well does the proposed modular, cloud-native EHR platform align with the specific needs and requirements outlined in the RFP?
  → What evidence or case studies can the vendor provide to demonstrate the successful implementation and adoption of similar solutions in other provinces?
  → How comprehensive and adaptable are the configurable workflows offered by the platform to cater to the diverse needs of clinics and hospitals?
🔢 Prompt: 289 tokens | Completion: 2 tokens | Cost: $0.0004 USD

🧠 Scoring Thought:
→ How well does the proposed modular, cloud-native EHR platform align with the specific needs and requirements outlined in the RFP?
📩 LLM Response: 8
✅ Parsed Score: 8/10
🔢 Prompt: 287 tokens | Completion: 2 tokens | Cost: $0.0004 USD

🧠 Scoring Thought:
→ What evidence or case studies can the vendor pr

# **Section 5: Print Results** <a id="5"></a>
**Display Evaluation Metrics**

In [15]:
print_proposal_evaluation(results_a, overall_score_a, swot_summary_a)



📊 PROPOSAL EVALUATION

🔹 Criterion: Solution Fit
📈 Score: 8/10
🧠 Thoughts:
   • How well does the proposed modular, cloud-native EHR platform align with the specific needs and requirements outlined in the RFP?
   • How does the modularity of the proposed EHR platform align with the scalability requirements outlined in the RFP? Are there specific modules that can be easily added or removed to accommodate future growth or changes in the healthcare environment?
🛠️ Tools Used:
   • suggest_tool_for["check_guideline"]: Best match based on your goal:
keyword_match_in_section (match: 0.34)
check_timeline_feasibility (match: 0.32)
check_summary_support (match: 0.31)
   • keyword_match_in_section["modularity and scalability"]: The keyword 'modularity and scalability' was NOT found in the section.
🗣️ Explanation: The proposal from Vendor A offers a modular, cloud-native EHR platform with configurable workflows that align well with the scalability requirements outlined in the RFP. The ability to